Code Author:
@Naman_Jaswani

# Algo for dataset labeling:

#### Lets take an example of   Attribute:  'Brand' , Category: 'Top Wear'

1. Loop over all rows unsup_data for top wear
2. Get description of i'th row and make it lower case
3. Initialize Tags for this i'th row as [O,O,O,O,O,,,,,O] with same length as length of tokenized description text.
4. Loop over all sample values for 'Brand'
5. If any token in description matches with this value, find and store its index.
6. Then still inside the same loop as(1), loop over all stored indices and label them. Consecutive labels will be marked as B_*, I_*.

In [ ]:
import numpy as np
import pandas as pd

from tqdm import tqdm

from nltk.tokenize import word_tokenize

import os

In [ ]:
os.listdir('../input/attributevalue2/')

In [ ]:
unsup = pd.read_csv('../input/complete-test-data/unsup_data.csv')
sample_test = pd.read_csv('../input/sample-data/test.csv')
sample_test_output = pd.read_csv('../input/sample-data/test_output.csv')
print(unsup.shape)
unsup.head()

# Top Wear

In [ ]:
unsup_top = unsup[unsup['Category']=='Top wear'].reset_index(drop='True')
unsup_top = unsup_top.dropna().reset_index(drop=True)

print(unsup_top.head())
print(unsup_top.shape[0])

In [ ]:
top_wear = pd.read_csv('../input/attributevalue2/Top Wear.csv')
top_wear['Sample values split'] = top_wear['Sample values'].apply(lambda x: x.split('||'))
dict_top_wear = dict(zip(top_wear['Standardized attributes'], top_wear['Sample values split']))


# top_wear
bottom_wear = pd.read_csv('../input/attributevalue2/Bottom Wear.csv')
bottom_wear['Sample values split'] = bottom_wear['Sample values'].apply(lambda x: x.split('||'))
dict_bottom_wear = dict(zip(bottom_wear['Standardized attributes'], bottom_wear['Sample values split']))

foot_wear = pd.read_csv('../input/attributevalue2/Foot Wear.csv')
foot_wear['Sample values split'] = foot_wear['Sample values'].apply(lambda x: x.split('||'))
dict_foot_wear = dict(zip(foot_wear['Standardized attributes'], foot_wear['Sample values split']))
# dict_bottom_wear['brand']

full_wear = pd.read_csv('../input/attributevalue2/Full Wear.csv')
full_wear['Sample values split'] = full_wear['Sample values'].apply(lambda x: x.split('||'))
dict_full_wear = dict(zip(full_wear['Standardized attributes'], full_wear['Sample values split']))
dict_full_wear['lining_material'] = dict_full_wear['lining material']
del dict_full_wear['lining material']

accessories = pd.read_csv('../input/attributevalue2/Accessories.csv')
accessories['Sample values split'] = accessories['Sample values'].apply(lambda x: x.split('||'))
dict_accessories = dict(zip(accessories['Standardized attributes'], accessories['Sample values split']))



In [ ]:
len(dict_bottom_wear['brand'])

## Brand

In [ ]:
# Actual code in deployment

t = 'B_brand'
t_ = 'I_brand'
att = 'brand'

unsup_top['Tag'] = 0

more_brands = ['ONL', 'Humor Gang', 'Celebravo','Zola','Stylum', 'Saibhya','DIVIE', 'hakashi','s v creation', 'FOREVER 21', 'TII','Morten Fashion']
recheck_brands = ['and','only', 'basics']

for brand in more_brands:
    dict_top_wear['brand'].append(brand) # adding another brand

for i in tqdm(range(unsup_top.shape[0])):
    dcrp = word_tokenize(unsup_top['Descriptions'][i].lower())
    unsup_top['Tag'][i] = ['O'] * len(dcrp)
    
    tag = unsup_top['Tag'][i].copy()
    all_indices = []
    
    dict_top_wear[att] = sorted(dict_top_wear[att], key=len, reverse=True)
    for value in dict_top_wear['brand']:
        if(value not in recheck_brands):
            value = word_tokenize(value.lower())
            if any(value==dcrp[ic:ic+len(value)] for ic in range(len(dcrp)-len(value)+1)):
                for ic in range(len(dcrp)-len(value)+1):
                    indices = []
                    if(value==dcrp[ic:ic+len(value)]):
                        for j in range(len(value)):
                            indices.append(ic+j)
                        indices.sort()
                        
                        for j in indices:
                            dcrp[j] = 'naman'
                        
                        if(len(indices)):
                            all_indices.append(indices)

            

    for indices_ in all_indices: # tagging
        flag = True
        for k in indices_:
            if(tag[k][0] != 'I' and not (tag[k][0]=='B' and k+1<len(tag) and tag[k+1][0]=='I')):
                if(flag):
                    tag[k] = t
                    flag = False
                else:
                    tag[k] = t_

        unsup_top['Tag'][i] = tag
        tag = unsup_top['Tag'][i].copy()

In [ ]:

singles = ['fabric_purity','hem','color_shade','color','detail_placement','ideal_for','collar','fit','occasion','sleeve_style','sleeve_length_type','size','pattern','ornamentation_type',
           'suitable_for','topwear_length_type','fabric_purity','fabric_care']

for att in tqdm(singles):
    # Actual code in deployment

    t = f'B_{att}'
    t_ = f'I_{att}'
    recheck = ['and','only', 'basics']
    
    if(att == 'brand'):
        more_brands = ['ONL', 'Humor Gang', 'Celebravo','Zola','Stylum', 'Saibhya','DIVIE', 'hakashi','s v creation', 'FOREVER 21', 'TII','Morten Fashion']
        recheck_brands = ['and','only', 'basics']
        
        for brand in more_brands:
            dict_top_wear[att].append(brand) # adding another brand

    for i in range(unsup_top.shape[0]):
        dcrp = word_tokenize(unsup_top['Descriptions'][i].lower())
        
        tag = unsup_top['Tag'][i].copy()
        all_indices = []
        
        dict_top_wear[att] = sorted(dict_top_wear[att], key=len, reverse=True)
        for value in dict_top_wear[att]:
            if((value not in recheck)):
                value = word_tokenize(value.lower())

                if any(value==dcrp[ic:ic+len(value)] for ic in range(len(dcrp)-len(value)+1)):
                    for ic in range(len(dcrp)-len(value)+1):
                        indices = []
                        if(value==dcrp[ic:ic+len(value)]):
                            for j in range(len(value)):
                                indices.append(ic+j)
                            indices.sort()

                            for j in indices:
                                dcrp[j] = 'naman'

                            if(len(indices)):
                                all_indices.append(indices)


        for indices_ in all_indices: # tagging
            flag = True
            for k in indices_:
                if(tag[k][0] != 'I' and not (tag[k][0]=='B' and k+1<len(tag) and tag[k+1][0]=='I')):
                    if(flag):
                        tag[k] = t
                        flag = False
                    else:
                        tag[k] = t_

            unsup_top['Tag'][i] = tag
            
#     first_att = False

In [ ]:
unsup_top.to_pickle('TOP WEAR.pickle')

# Bottom Wear

In [ ]:
unsup['Category'].unique()

In [ ]:
unsup_bottom = unsup[unsup['Category']=='Bottom Wear'].reset_index(drop='True')
unsup_bottom = unsup_bottom.dropna().reset_index(drop=True)
unsup_bottom.shape

In [ ]:
unsup_bottom['Tag'] = 0

first_att = True

singles = ['trouser_occasion','color','jean_bottomwear_length_type','ideal_for','suitable_for','jean_ornamentation_type','trouser_pleats',
          'rise','size','jean_wash','pattern','jean_pocket_type','fit','jean_faded','closure','brand','fabric_care', 'fabric']

for att in tqdm(singles):
    # Actual code in deployment

    t = f'B_{att}'
    t_ = f'I_{att}'
    recheck = ['and','only', 'basics']
    
    if(att == 'brand'):
        more_brands = ['ONL', 'Humor Gang', 'Celebravo','Zola','Stylum', 'Saibhya','DIVIE', 'hakashi','s v creation', 'FOREVER 21', 'TII','Morten Fashion']
        recheck_brands = ['and','only', 'basics']
        
        for brand in more_brands:
            dict_bottom_wear[att].append(brand) # adding another brand

    for i in range(unsup_bottom.shape[0]):
        dcrp = word_tokenize(unsup_bottom['Descriptions'][i].lower())
        
        if(first_att):
            unsup_bottom['Tag'][i] = ['O'] * len(dcrp)
        
        tag = unsup_bottom['Tag'][i].copy()
        all_indices = []
        
        dict_bottom_wear[att] = sorted(dict_bottom_wear[att], key=len, reverse=True)
        for value in dict_bottom_wear[att]:
            if((value not in recheck)):
                value = word_tokenize(value.lower())

                if any(value==dcrp[ic:ic+len(value)] for ic in range(len(dcrp)-len(value)+1)):
                    for ic in range(len(dcrp)-len(value)+1):
                        indices = []
                        if(value==dcrp[ic:ic+len(value)]):
                            for j in range(len(value)):
                                indices.append(ic+j)
                            indices.sort()

                            for j in indices:
                                dcrp[j] = 'naman'

                            if(len(indices)):
                                all_indices.append(indices)


        for indices_ in all_indices: # tagging
            flag = True
            for k in indices_:
                if(tag[k][0] != 'I' and not (tag[k][0]=='B' and k+1<len(tag) and tag[k+1][0]=='I')):
                    if(flag):
                        tag[k] = t
                        flag = False
                    else:
                        tag[k] = t_

            unsup_bottom['Tag'][i] = tag
            
    first_att = False

In [ ]:
unsup_bottom.to_pickle('BOTTOM WEAR.pickle')

# Foot Wear

In [ ]:
unsup['Category'].unique()

In [ ]:
unsup_foot = unsup[unsup['Category']=='Foot Wear'].reset_index(drop='True')
unsup_foot = unsup_foot.dropna().reset_index(drop=True)
unsup_foot.shape

In [ ]:
unsup_foot['Tag'] = 0
for i in range(unsup_foot.shape[0]):
    dcrp = word_tokenize(unsup_foot['Descriptions'][i].lower())
    unsup_foot['Tag'][i] = ['O'] * len(dcrp)
            
singles = ['shoe_detail_placement','leather_type','occasion','ornamentation_type','footwear_pattern_placement','shoe_tip_shape',
           'shoe_color_shade','color','ideal_for','sole_material',
          'size','pattern','closure','brand']

for att in tqdm(singles):
    # Actual code in deployment

    t = f'B_{att}'
    t_ = f'I_{att}'
    recheck = ['and', 'only', 'basics']
    
    if(att == 'brand'):
        more_brands = ['ONL', 'Humor Gang', 'Celebravo','Zola','Stylum', 'Saibhya','DIVIE', 'hakashi','s v creation', 'FOREVER 21', 'TII','Morten Fashion']
        recheck_brands = ['and', 'only', 'basics']
        
        for brand in more_brands:
            dict_foot_wear[att].append(brand) # adding another brand

    for i in range(unsup_foot.shape[0]):
        dcrp = word_tokenize(unsup_foot['Descriptions'][i].lower()).copy()

        
        tag = unsup_foot['Tag'][i].copy()
        all_indices = []
        dict_foot_wear[att] = sorted(dict_foot_wear[att], key=len, reverse=True)
        
        for value in dict_foot_wear[att]:
            if((value not in recheck)):
                value = word_tokenize(value.lower())

                if any(value==dcrp[ic:ic+len(value)] for ic in range(len(dcrp)-len(value)+1)):
                    for ic in range(len(dcrp)-len(value)+1):
                        indices = []
                        if(value==dcrp[ic:ic+len(value)]):
                            for j in range(len(value)):
                                indices.append(ic+j)
                            indices.sort()

                            for j in indices:
                                dcrp[j] = 'naman'

                            if(len(indices)):
                                all_indices.append(indices)


        for indices_ in all_indices: # tagging
            flag = True
            for k in indices_:
                if(tag[k][0] != 'I' and not (tag[k][0]=='B' and k+1<len(tag) and tag[k+1][0]=='I')):
                    if(flag):
                        tag[k] = t
                        flag = False
                    else:
                        tag[k] = t_

            unsup_foot['Tag'][i] = tag
            
#     first_att = False

## Labeling Ambiguous FOOT WEAR

In [ ]:
atts = ['inner_material','outer_material','sole_material']

for i in range(unsup_foot.shape[0]):
    dcrp = word_tokenize(unsup_foot['Descriptions'][i].lower())
    tag = unsup_foot['Tag'][i].copy()
    
    all_lst = []
    
    attr_name_indices = []
    attr_names = []
    
    v = ['inner']
    if any(v==dcrp[i:i+len(v)] for i in range(len(dcrp)-len(v)+1)):
        all_lst.append('inner')
        att_name = 'inner_material'
        attr_name_indices.append(dcrp.index(v[0]))
        attr_names.append(att_name)
        
    v = ['outer']
    if any(v==dcrp[i:i+len(v)] for i in range(len(dcrp)-len(v)+1)):
        all_lst.append('outer')
        att_name = 'outer_material'
        attr_name_indices.append(dcrp.index(v[0]))
        attr_names.append(att_name)
        
    v = ['sole']
    if any(v==dcrp[i:i+len(v)] for i in range(len(dcrp)-len(v)+1)):
        all_lst.append('sole')
        att_name = 'sole_material'
        attr_name_indices.append(dcrp.index(v[0]))
        attr_names.append(att_name)
    
    all_set = set(all_lst)

    if(len(all_set)==0):
        continue
    
    if(len(all_set)==1): # ----------------- Single ambiguous attribute ---------------
        attr = list(all_set)[0]
        if(attr=='inner'):
            t = "B_inner_material"
            t_ = "I_inner_material"
        
        if(attr=='outer'):
            t = "B_outer_material"
            t_ = "I_outer_material"
        
        if(attr=='sole'):
            t = "B_sole_material"
            t_ = "I_sole_material"
        
        dcrp = word_tokenize(unsup_foot['Descriptions'][i].lower())
        
        tag = unsup_foot['Tag'][i].copy()
        all_indices = []
        dict_foot_wear[att] = sorted(dict_foot_wear[att], key=len, reverse=True)
        
        for value in dict_foot_wear[att_name]:
            value = word_tokenize(value.lower())
            if any(value==dcrp[ic:ic+len(value)] for ic in range(len(dcrp)-len(value)+1)):
                for ic in range(len(dcrp)-len(value)+1):
                    indices = []
                    if(value==dcrp[ic:ic+len(value)]):
                        for j in range(len(value)):
                            indices.append(ic+j)
                        indices.sort()

                        for j in indices:
                            dcrp[j] = 'naman'

                        if(len(indices)):
                            all_indices.append(indices)
        
        for indices_ in all_indices: # tagging
            flag = True
            for k in indices_:
                if(tag[k][0] != 'I' and not (tag[k][0]=='B' and k+1<len(tag) and tag[k+1][0]=='I')):
                    if(flag):
                        tag[k] = t
                        flag = False
                    else:
                        tag[k] = t_

            unsup_foot['Tag'][i] = tag
            
    if(len(all_set) > 1): #---------------- Multiple ambiguous attribute occurances ---------------
        
        for ix in range(len(attr_name_indices)):
            att_index = attr_name_indices[ix]
            att_name = attr_names[ix]
            if(att_name=='inner_material'):
                t = "B_inner_material"
                t_ = "I_inner_material"
        
            if(att_name=='outer_material'):
                t = "B_outer_material"
                t_ = "I_outer_material"

            if(att_name=='sole_material'):
                t = "B_sole_material"
                t_ = "I_sole_material"
            
            done = False
            
            for x in [1, 2, 3]:
            # Bi-gram analysis
                all_indices = []
                dict_foot_wear[att] = sorted(dict_foot_wear[att], key=len, reverse=True)
                
                for value in dict_foot_wear[att_name]:
                    value = word_tokenize(value.lower())
                    indices = []
                    if (value==dcrp[att_index+x:att_index+x+len(value)] or
                            value==dcrp[att_index-x-len(value)+1:att_index-x+1]):
                        
                        done = True
                        for j in range(len(value)):
                            if(value==dcrp[att_index+x:att_index+x+len(value)]):
                                found_idx = att_index+x
                            else:
                                found_idx = att_index-x-len(value)+1
                            if(np.abs(found_idx - att_index) <= 3):
                                indices.append(found_idx)
                                indices.sort()

                    for j in indices:
                        dcrp[j] = 'naman'

                    if(len(indices)):
                        all_indices.append(indices)
                
#                 print(t)
                for indices_ in all_indices: # tagging
                    flag = True
                    for k in indices_:
                        if(tag[k][0] != 'I' and not (tag[k][0]=='B' and k+1<len(tag) and tag[k+1][0]=='I')):
                            if(flag):
                                tag[k] = t
                                flag = False
                            else:
                                tag[k] = t_

                    unsup_foot['Tag'][i] = tag
                    tag = unsup_foot['Tag'][i].copy()

                if(done):
                    break

In [ ]:
unsup_foot.to_pickle('FOOT WEAR.pickle')

# Full Wear

In [ ]:
unsup['Category'].unique()

In [ ]:
unsup_full = unsup[unsup['Category']=='Full wear'].reset_index(drop='True')
unsup_full = unsup_full.dropna().reset_index(drop=True)
unsup_full

In [ ]:
unsup_full['Tag'] = 0
for i in range(unsup_full.shape[0]):
    dcrp = word_tokenize(unsup_full['Descriptions'][i].lower())
    unsup_full['Tag'][i] = ['O'] * len(dcrp)

singles = ['sari_blouse_stitch_type','gown_weave_type','gown_belt_material','sari_border_details','sari_border_length','brand','gown_bust_size','gown_closure','color','gown_design','detail_placement','sari_embroidery_method','sari_type_of_embroidery','ideal_for','dress_topwear_length_type',
          'occasion','ornamentation_type','pattern','sari_fabric_purity','dress_shape','size','sleeve_length_type','sleeve_style','collar','fabric_care','sari_decorative_material','fabric']

for att in tqdm(singles):
    # Actual code in deployment

    t = f'B_{att}'
    t_ = f'I_{att}'
    recheck = ['and','only', 'basics']
    
    if(att == 'brand'):
        more_brands = ['ONL', 'Humor Gang', 'Celebravo','Zola','Stylum', 'Saibhya','DIVIE', 'hakashi','s v creation', 'FOREVER 21', 'TII','Morten Fashion']
        recheck_brands = ['and','only', 'basics']
        
        for brand in more_brands:
            dict_full_wear[att].append(brand) # adding another brand

    for i in range(unsup_full.shape[0]):
        dcrp = word_tokenize(unsup_full['Descriptions'][i].lower())
        
        tag = unsup_full['Tag'][i].copy()
        all_indices = []
        dict_full_wear[att] = sorted(dict_full_wear[att], key=len, reverse=True)
        
        for value in dict_full_wear[att]:
            if((value not in recheck)):
                value = word_tokenize(value.lower())

                if any(value==dcrp[ic:ic+len(value)] for ic in range(len(dcrp)-len(value)+1)):
                    for ic in range(len(dcrp)-len(value)+1):
                        indices = []
                        if(value==dcrp[ic:ic+len(value)]):
                            for j in range(len(value)):
                                indices.append(ic+j)
                            indices.sort()

                            for j in indices:
                                dcrp[j] = 'naman'

                            if(len(indices)):
                                all_indices.append(indices)

        for indices_ in all_indices: # tagging
            flag = True
            for k in indices_:
                if(tag[k][0] != 'I' and not (tag[k][0]=='B' and k+1<len(tag) and tag[k+1][0]=='I')):
                    if(flag):
                        tag[k] = t
                        flag = False
                    else:
                        tag[k] = t_

            unsup_full['Tag'][i] = tag
            tag = unsup_full['Tag'][i].copy()
            

## Labeling Ambiguous Full Wear

In [ ]:
atts = ['sari_blouse_fabric','lining_material']

for i in range(unsup_full.shape[0]):
    dcrp = word_tokenize(unsup_full['Descriptions'][i].lower())
    tag = unsup_full['Tag'][i].copy()
    
    all_lst = []
    
    attr_name_indices = []
    attr_names = []
    
    v = ['blouse']
    if any(v==dcrp[i:i+len(v)] for i in range(len(dcrp)-len(v)+1)):
        all_lst.append('blouse')
        att_name = 'sari_blouse_fabric'
        attr_name_indices.append(dcrp.index(v[0]))
        attr_names.append(att_name)

        
    v = ['lining']
    if any(v==dcrp[i:i+len(v)] for i in range(len(dcrp)-len(v)+1)):
        all_lst.append('lining')
        att_name = 'lining_material'
        attr_name_indices.append(dcrp.index(v[0]))
        attr_names.append(att_name)
    
    all_set = set(all_lst)

    if(len(all_set)==0):
        continue
    
    if(len(all_set)==1): # ----------------- Single ambiguous attribute ---------------
        attr = list(all_set)[0]
        if(attr=='blouse'):
            t = "B_sari_blouse_fabric"
            t_ = "I_sari_blouse_fabric"

        
        if(attr=='lining'):
            t = "B_lining_material"
            t_ = "I_lining_material"
        
        dcrp = word_tokenize(unsup_full['Descriptions'][i].lower())
        
        tag = unsup_full['Tag'][i].copy()
        all_indices = []
        dict_full_wear[att] = sorted(dict_full_wear[att], key=len, reverse=True)
        
        for value in dict_full_wear[att_name]:
            value = word_tokenize(value.lower())

            if any(value==dcrp[ic:ic+len(value)] for ic in range(len(dcrp)-len(value)+1)):
                for ic in range(len(dcrp)-len(value)+1):
                    indices = []
                    if(value==dcrp[ic:ic+len(value)]):
                        for j in range(len(value)):
                            indices.append(ic+j)
                        indices.sort()

                        for j in indices:
                            dcrp[j] = 'naman'

                        if(len(indices)):
                            all_indices.append(indices)
        
        for indices_ in all_indices: # tagging
            flag = True
            for k in indices_:
                if(tag[k][0] != 'I' and not (tag[k][0]=='B' and k+1<len(tag) and tag[k+1][0]=='I')):
                    if(flag):
                        tag[k] = t
                        flag = False
                    else:
                        tag[k] = t_

            unsup_full['Tag'][i] = tag
            tag = unsup_full['Tag'][i].copy()
        
    else: #---------------- Multiple ambiguous attribute occurances ---------------
        for ix in range(len(attr_name_indices)):
            att_index = attr_name_indices[ix]
            att_name = attr_names[ix]
#             print(att_name)
            
            if(attr=='blouse'):
                t = "B_sari_blouse_fabric"
                t_ = "I_sari_blouse_fabric"


            if(attr=='lining'):
                t = "B_lining_material"
                t_ = "I_lining_material"
            
            done = False
            for x in [1, 2]:
            # Bi-gram analysis
                all_indices = []
                dict_full_wear[att] = sorted(dict_full_wear[att], key=len, reverse=True)
                for value in dict_full_wear[att_name]:
                    value = word_tokenize(value.lower())
                    indices = []
                    if (value==dcrp[att_index+x:att_index+x+len(value)] or
                            value==dcrp[att_index-x-len(value)+1:att_index-x+1]):
                        
                        done = True
                        for j in range(len(value)):
                            if(value==dcrp[att_index+x:att_index+x+len(value)]):
                                found_idx = att_index+x
                            else:
                                found_idx = att_index-x-len(value)+1
                                
#                             print(att_index, found_idx)
                            if(np.abs(found_idx - att_index) <= 3):
                                indices.append(found_idx)
                                indices.sort()

                    for j in indices:
                        dcrp[j] = 'naman'

                    if(len(indices)):
                        all_indices.append(indices)
                
                all_indices.sort()
                for indices_ in all_indices: # tagging
                    flag = True
                    for k in indices_:
                        if(tag[k][0] != 'I' and not (tag[k][0]=='B' and k+1<len(tag) and tag[k+1][0]=='I')):
                            if(flag):
                                tag[k] = t
                                flag = False
                            else:
                                tag[k] = t_

                    unsup_full['Tag'][i] = tag
                    tag = unsup_full['Tag'][i].copy()

                if(done):
                    break

In [ ]:
unsup_full.to_pickle('FULL WEAR.pickle')

# Accessories

In [ ]:
unsup['Category'].unique()

In [ ]:
unsup_acc = unsup[unsup['Category']=='Accessories'].reset_index(drop='True')
unsup_acc = unsup_acc.dropna().reset_index(drop=True)
unsup_acc

In [ ]:
unsup_acc['vertical'].unique()

## EDA for accessories

In [ ]:

for i in range(unsup_acc.shape[0]):
    dcrp = word_tokenize(unsup_acc['Descriptions'][i].lower())
    
    all_lst = []
    
    attr_name_indices = []
    attr_names = []
    
    v = ['watch']
    if any(v==dcrp[i:i+len(v)] for i in range(len(dcrp)-len(v)+1)):
        all_lst.append('watch')
        att_name = 'watch'
        attr_name_indices.append(dcrp.index(v[0]))
        attr_names.append(att_name)
        
    v = ['suitcase']
    if any(v==dcrp[i:i+len(v)] for i in range(len(dcrp)-len(v)+1)):
        all_lst.append('suitcase')
        att_name = 'suitcase'
        attr_name_indices.append(dcrp.index(v[0]))
        attr_names.append(att_name)
        
    v = ['jewellery']
    if any(v==dcrp[i:i+len(v)] for i in range(len(dcrp)-len(v)+1)):
        all_lst.append('jewellery')
        att_name = 'jewellery'
        attr_name_indices.append(dcrp.index(v[0]))
        attr_names.append(att_name)
        
    v = ['bag']
    if any(v==dcrp[i:i+len(v)] for i in range(len(dcrp)-len(v)+1)):
        all_lst.append('bag')
        att_name = 'bag'
        attr_name_indices.append(dcrp.index(v[0]))
        attr_names.append(att_name)
    
    v = ['sunglasses']
    if any(v==dcrp[i:i+len(v)] for i in range(len(dcrp)-len(v)+1)):
        all_lst.append('sunglasses')
        att_name = 'sunglasses'
        attr_name_indices.append(dcrp.index(v[0]))
        attr_names.append(att_name)
    
    all_set = set(all_lst)
    
    

## Labeling General Accessories [att]

In [ ]:
unsup_acc['Tag'] = 0
for i in range(unsup_acc.shape[0]):
    dcrp = word_tokenize(unsup_acc['Descriptions'][i].lower())
    unsup_acc['Tag'][i] = ['O'] * len(dcrp)

singles = ['size','color','occasion','material','ideal_for','shape','closure','brand']

for att in tqdm(singles):
    # Actual code in deployment

    t = f'B_{att}'
    t_ = f'I_{att}'
    recheck = ['and','only', 'basics']
    
    if(att == 'brand'):
        more_brands = ['ONL', 'Humor Gang', 'Celebravo','Zola','Stylum', 'Saibhya','DIVIE', 'hakashi','s v creation', 'FOREVER 21', 'TII','Morten Fashion']
        recheck_brands = ['and','only', 'basics']
        
        for brand in more_brands:
            dict_accessories[att].append(brand) # adding another brand

    for i in range(unsup_acc.shape[0]):
        dcrp = word_tokenize(unsup_acc['Descriptions'][i].lower())
        
        tag = unsup_acc['Tag'][i].copy()
        all_indices = []
        
        dict_accessories[att] = sorted(dict_accessories[att], key=len, reverse=True)
        for value in dict_accessories[att]:
            if((value not in recheck)):
                value = word_tokenize(value.lower())

                if any(value==dcrp[ic:ic+len(value)] for ic in range(len(dcrp)-len(value)+1)):
                    for ic in range(len(dcrp)-len(value)+1):
                        indices = []
                        if(value==dcrp[ic:ic+len(value)]):
                            for j in range(len(value)):
                                indices.append(ic+j)
                            indices.sort()

                            for j in indices:
                                dcrp[j] = 'naman'

                            if(len(indices)):
                                all_indices.append(indices)

        for indices_ in all_indices: # tagging
            flag = True
            for k in indices_:
                if(tag[k][0] != 'I' and not (tag[k][0]=='B' and k+1<len(tag) and tag[k+1][0]=='I')):
                    if(flag):
                        tag[k] = t
                        flag = False
                    else:
                        tag[k] = t_

            unsup_acc['Tag'][i] = tag
            tag = unsup_acc['Tag'][i].copy()
            

In [ ]:
def general(singles, dcrp, test_tag):
    
    for att in (singles):
        t = f'B_{att}'
        t_ = f'I_{att}'
        recheck = ['and','only', 'basics']

        if(att == 'brand'):
            more_brands = ['ONL', 'Humor Gang', 'Celebravo','Zola','Stylum', 'Saibhya','DIVIE', 'hakashi','s v creation', 'FOREVER 21', 'TII','Morten Fashion']
            recheck_brands = ['and','only', 'basics', 'high quality']

            for brand in more_brands:
                dict_accessories[att].append(brand) # adding another brand

        tag = test_tag.copy()
        all_indices = []

        dict_accessories[att] = sorted(dict_accessories[att], key=len, reverse=True)
        for value in dict_accessories[att]:
            if((value not in recheck)):
                value = word_tokenize(value.lower())

                if any(value==dcrp[ic:ic+len(value)] for ic in range(len(dcrp)-len(value)+1)):
                    for ic in range(len(dcrp)-len(value)+1):
                        indices = []
                        if(value==dcrp[ic:ic+len(value)]):
                            for j in range(len(value)):
                                indices.append(ic+j)
                            indices.sort()

                            for j in indices:
                                dcrp[j] = 'naman'

                            if(len(indices)):
                                all_indices.append(indices)

        for indices_ in all_indices: # tagging
            flag = True
            for k in indices_:
                if(tag[k][0] != 'I' and not (tag[k][0]=='B' and k+1<len(tag) and tag[k+1][0]=='I')):
                    if(flag):
                        tag[k] = t
                        flag = False
                    else:
                        tag[k] = t_

            test_tag = tag
            tag = test_tag.copy()
        
    return tag

In [ ]:
def ambiguous_case(singles, dcrp, test_tag, v_s):

    tag = test_tag.copy()
    
    all_lst = []
    
    attr_name_indices = []
    attr_names = []
    
    for v in v_s:
        idx = v_s.index(v)
        attr = singles[idx]
        
        if(len(v)>1):
            for vv in v:
                if any(vv==dcrp[i:i+len(vv)] for i in range(len(dcrp)-len(vv)+1)):
                    all_lst.append(vv[0])
                    att_name = attr
                    attr_name_indices.append(dcrp.index(vv[0]))
                    attr_names.append(att_name)
        else:
            if any(v==dcrp[i:i+len(v)] for i in range(len(dcrp)-len(v)+1)):
                all_lst.append(v[0])
                att_name = attr
                attr_name_indices.append(dcrp.index(v[0]))
                attr_names.append(att_name)
        
    
    all_set = set(all_lst)

    if(len(all_set)==0):
        return tag
    
    if(len(all_set)==1): # ----------------- Single ambiguous attribute ---------------
        t = f'B_{att_name}'
        t_ = f'I_{att_name}'        
        
        tag = test_tag.copy()
        all_indices = []
        dict_accessories[att] = sorted(dict_accessories[att], key=len, reverse=True)
        
        for value in dict_accessories[att_name]:
            value = word_tokenize(value.lower())

            if any(value==dcrp[ic:ic+len(value)] for ic in range(len(dcrp)-len(value)+1)):
                for ic in range(len(dcrp)-len(value)+1):
                    indices = []
                    if(value==dcrp[ic:ic+len(value)]):
                        for j in range(len(value)):
                            indices.append(ic+j)
                        indices.sort()

                        for j in indices:
                            dcrp[j] = 'naman'

                        if(len(indices)):
                            all_indices.append(indices)
        
        for indices_ in all_indices: # tagging
            flag = True
            for k in indices_:
                if(tag[k][0] != 'I' and not (tag[k][0]=='B' and k+1<len(tag) and tag[k+1][0]=='I')):
                    if(flag):
                        tag[k] = t
                        flag = False
                    else:
                        tag[k] = t_

            test_tag = tag
            tag = test_tag.copy()
        
            
    else: #---------------- Multiple ambiguous attribute occurances ---------------
        for ix in range(len(attr_name_indices)):
            att_index = attr_name_indices[ix]
            att_name = attr_names[ix]
            
            t = f'B_{att_name}'
            t_ = f'I_{att_name}'
            
            done = False
            for x in [1, 2, 3]:
            # Bi-gram analysis
                all_indices = []
                dict_accessories[att] = sorted(dict_accessories[att], key=len, reverse=True)
                
                for value in dict_accessories[att_name]:
                    value = word_tokenize(value.lower())
                    indices = []
                    if (value==dcrp[att_index+x:att_index+x+len(value)] or
                            value==dcrp[att_index-x-len(value)+1:att_index-x+1]):
                        
                        done = True
                        for j in range(len(value)):
                            if(value==dcrp[att_index+x:att_index+x+len(value)]):
                                found_idx = att_index+x
                            else:
                                found_idx = att_index-x-len(value)+1
                                
                            if(np.abs(found_idx - att_index) <= 3):
                                indices.append(found_idx)
                                indices.sort()

                    for j in indices:
                        dcrp[j] = 'naman'

                    if(len(indices)):
                        all_indices.append(indices)
                
                all_indices.sort()
                for indices_ in all_indices: # tagging
                    flag = True
                    for k in indices_:
                        if(tag[k][0] != 'I' and not (tag[k][0]=='B' and k+1<len(tag) and tag[k+1][0]=='I')):
                            if(flag):
                                tag[k] = t
                                flag = False
                            else:
                                tag[k] = t_

                    test_tag = tag
                    tag = test_tag.copy()

                if(done):
                    break

    return tag

In [ ]:

for i in tqdm(range(unsup_acc.shape[0])):
    dcrp = word_tokenize(unsup_acc['Descriptions'][i].lower())
    tag = unsup_acc['Tag'][i].copy()
    
    if('sunglasses' in dcrp or 'sunglasses' in dcrp or 'glasses' in dcrp):
        # General labels
        singles = ['sunglass_purpose','sunglass_face_type','sunglass_frame','sunglass_design']
        unsup_acc['Tag'][i] = general(singles, dcrp, tag)
        
        #material
        abmiguous = ['sunglass_frame_material','sunglass_lens_material']
        tag = unsup_acc['Tag'][i].copy()
        dcrp = word_tokenize(unsup_acc['Descriptions'][i].lower())
        v_s = [[['frame'],['frames']], [['lens'],['lenses']]]
        unsup_acc['Tag'][i] = ambiguous_case(abmiguous, dcrp, tag, v_s)
        
        #color
        abmiguous = ['frame_color','sunglass_lens_color']
        tag = unsup_acc['Tag'][i].copy()
        dcrp = word_tokenize(unsup_acc['Descriptions'][i].lower())
        v_s = [[['frame'],['frames']], [['lens'],['lenses']]]
        unsup_acc['Tag'][i] = ambiguous_case(abmiguous, dcrp, tag, v_s)
        
    
    if('suitcase' in dcrp or 'bag' in dcrp):
        # General labels
        singles = ['strap_material','suitcase_detail_placement','bag_leather_type','suitcase_ornamentation_type','pattern','strap_color']
        unsup_acc['Tag'][i] = general(singles, dcrp, tag)
        
    if('watch' in dcrp or 'clock' in dcrp):
        # General labels
        singles = ['watch_dial_shape','watch_strap_design']
        unsup_acc['Tag'][i] = general(singles, dcrp, tag)
        
        #material
        abmiguous = ['watch_case_bezel_material','strap_material']
        tag = unsup_acc['Tag'][i].copy()
        dcrp = word_tokenize(unsup_acc['Descriptions'][i].lower())
        v_s = [[['case'],['bezel']], ['strap']]
        unsup_acc['Tag'][i] = ambiguous_case(abmiguous, dcrp, tag, v_s)
        
        #color
        abmiguous = ['dial_color','strap_color']
        tag = unsup_acc['Tag'][i].copy()
        dcrp = word_tokenize(unsup_acc['Descriptions'][i].lower())
        v_s = [['dial'], ['strap']]
        unsup_acc['Tag'][i] = ambiguous_case(abmiguous, dcrp, tag, v_s)
        
    if('jewellery' in dcrp or 'jewel' in dcrp):
        # General labels
        singles = ['jewellery_set_finish','jewellery_set_earring_length']
        unsup_acc['Tag'][i] = general(singles, dcrp, tag)
        
        #material
        abmiguous = ['jewellery_set_base_material','jewellery_set_artificial_pearl_material','jewellery_set_diamond_shape','jewellery_set_gemstone']
        tag = unsup_acc['Tag'][i].copy()
        dcrp = word_tokenize(unsup_acc['Descriptions'][i].lower())
        v_s = [[['case'],['bezel'],['body']], ['artificial pearl'],['diamond'], ['gemstone']]
        unsup_acc['Tag'][i] = ambiguous_case(abmiguous, dcrp, tag, v_s)
        
        #color
        abmiguous = ['jewellery_set_metal_color','jewellery_set_emerald_color','jewellery_set_pearl_color','jewellery_set_ruby_color','jewellery_set_sapphire_color','jewellery_set_silver_color']
        tag = unsup_acc['Tag'][i].copy()
        dcrp = word_tokenize(unsup_acc['Descriptions'][i].lower())
        v_s = [['metal'], ['emerald'], ['pearl'],['ruby'],['sapphire'],['silver']]
        unsup_acc['Tag'][i] = ambiguous_case(abmiguous, dcrp, tag, v_s)
        

In [ ]:
unsup_acc.to_pickle('ACCESSORIES.pickle')